# Saving a model and Callbacks

In [3]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Loading dataset & building model 

### Dataset

In [4]:
fashion_mnist = keras.datasets.fashion_mnist

In [5]:
## Validation Dataset is missing

In [6]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

In [7]:
x_valid, x_train = x_train[:5000], x_train[5000:]

In [8]:
# Creating the validation dataset from the training data. We use 5000 the rest will be used for training. 

In [9]:
y_valid, y_train = y_train[:5000], y_train[5000:]

In [10]:
## We should scale our data because we're gonna use Gradient Descent.

In [11]:
x_valid, x_train, x_test = x_valid/255, x_train/255, x_test/255

In [12]:
# Label-Map for Y

In [13]:
class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

### Creating the sequential model

In [14]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28,28]),
    keras.layers.Dense(300, activation = keras.activations.relu),
    keras.layers.Dense(100, activation = keras.activations.relu),
    keras.layers.Dense(10, activation = keras.activations.softmax)
])

In [15]:
model.compile(loss = keras.losses.sparse_categorical_crossentropy, optimizer = keras.optimizers.SGD(learning_rate=0.01), metrics = ["accuracy"])

## Training and ```fit()```

## ```Keras.model.save()```

keras provides an own function for saving model. Each keras.Model class provides a save method to save the model in h5 format: 

What will be saved? 
1. Model's architectrure
2. Hyperparamters
3. Model Parameters + Weight
4. Optimizer + Hyperparamter

In [22]:
# Keras saves model in h5 format ' it's easy

In [23]:
model.fit(x_train, y_train, epochs = 2, validation_data = (x_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3957 - accuracy: 0.8615 - val_loss: 0.3935 - val_accuracy: 0.8612
Epoch 2/2
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3808 - accuracy: 0.8656 - val_loss: 0.3701 - val_accuracy: 0.8748


In [24]:
model.save("models/firstModel.h5")

## Using Callbacks

The ```fit()``` methods allows us to provide a callback function, which is executed when: 

- Start & end of the training
- start & end of each poch
- Before & after processing batch

Will be used to store checkpoint files. __The keras.callbacks API already provides some common callback functions!__

In [28]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("models/firstModel_checkpoint.h5")

In [29]:
history = model.fit(x_train, y_train, epochs = 2, callbacks = [checkpoint_cb])

Epoch 1/2
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3467 - accuracy: 0.8766
Epoch 2/2
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3360 - accuracy: 0.8799


### ```save_best_only```

To keep only the model with the best validation score we can also provide the ```save_best_only=True``` argument to ModelCheckpoint-Callback. 

In [31]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("models/firstModel_checkpoint.h5", save_best_only = True)

In [32]:
history = model.fit(x_train, y_train, epochs = 2, callbacks = [checkpoint_cb])

Epoch 1/2
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3286 - accuracy: 0.8833
Epoch 2/2
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3202 - accuracy: 0.8865


### Early Stopping

Will interrupt if no progress is measured on the validation set

In [33]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights = True)

In [34]:
history = model.fit(x_train, y_train, epochs = 2, callbacks = [checkpoint_cb, early_stopping_cb])

Epoch 1/2
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3131 - accuracy: 0.8889
Epoch 2/2
1719/1719 [==============================] - 2s 1ms/step - loss: 0.3062 - accuracy: 0.8910


## Writing your own callback

Basically we are creating our own callback by overwritting specific function of the Callback class. 

Template Methods are: 

1. ```on_epoch_end()```
2. ```on_train_begin()```
3. ```on_batch_end()```
4. ...

In [38]:
class MyCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("Epoch finished!")

In [39]:
history = model.fit(x_train, y_train, epochs = 2, callbacks = [MyCallback()])

Epoch 1/2
1719/1719 [==============================] - 2s 1ms/step - loss: 0.2927 - accuracy: 0.8960
Epoch finished!
Epoch 2/2
1719/1719 [==============================] - 2s 1ms/step - loss: 0.2875 - accuracy: 0.8962
Epoch finished!
